In [1]:
%pip install pandas

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ------------- -------------------------- 3.7/11.0 MB 21.0 MB/s eta 0:00:01
   ----------------------------- ---------- 8.1/11.0 MB 21.0 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 20.6 MB/s  0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ------------ --------------------------- 3.9/12.8 MB 19.3 MB/s eta 0:00:01
   ----------------------- ---------------- 7.6/12.8 MB 20.8 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 20.5 MB/s  0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/4 [pytz]
   ---------- ----------------------------- 1/4 [tzdata]
   -------------------- --------------

In [5]:
%pip install tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import sys

def process_patient_data(csv_file):
    """
    Memuat data pasien dari CSV, melakukan tokenisasi gejala,
    dan memfilter pasien berdasarkan kriteria yang ditentukan.
    
    Kriteria Filter:
    1. Usia > 40 tahun
    2. Jumlah gejala > 2
    """
    try:
        # 1. Memuat CSV
        # Menggunakan skipinitialspace=True untuk menangani spasi 
        # yang mungkin ada setelah koma pemisah
        df = pd.read_csv(csv_file, skipinitialspace=True)
        
        # Memastikan tipe data 'age' adalah numerik
        df['age'] = pd.to_numeric(df['age'])

        print("--- Data Asli (DataFrame) ---")
        print(df.to_markdown(index=False))
        print("-" * 70)

        # 2. Tokenisasi gejala menjadi list
        #    - .split(',') membagi string berdasarkan koma
        #    - [s.strip() for s in ...] membersihkan spasi di awal/akhir
        #      setiap gejala (contoh: " batuk" menjadi "batuk")
        df['symptom_list'] = df['symptoms'].apply(lambda x: [s.strip() for s in x.split(',')])
        
        # Membuat kolom baru untuk jumlah gejala
        df['symptom_count'] = df['symptom_list'].apply(len)

        print("--- Data Setelah Tokenisasi Gejala ---")
        print(df.to_markdown(index=False))
        print("-" * 70)

        # 3. Filter pasien
        filter_usia = df['age'] > 40
        filter_gejala = df['symptom_count'] > 2

        # Menggabungkan kedua kondisi filter (AND)
        filtered_patients = df[filter_usia & filter_gejala]

        return filtered_patients

    except FileNotFoundError:
        print(f"Error: File '{csv_file}' tidak ditemukan.", file=sys.stderr)
        return pd.DataFrame() # Mengembalikan DataFrame kosong
    except Exception as e:
        print(f"Terjadi error saat pemrosesan data: {e}", file=sys.stderr)
        return pd.DataFrame()

In [2]:
FILE_PASIEN = 'patients.csv'

print(f"Memulai pemrosesan untuk file: {FILE_PASIEN}\n")

hasil_filter = process_patient_data(FILE_PASIEN)

if not hasil_filter.empty:
    print("\n--- Hasil Filter (Usia > 40 TAHUN & Jumlah Gejala > 2) ---")
    # Menampilkan hasil filter dengan format yang lebih rapi
    print(hasil_filter.to_markdown(index=False))
else:
    print("\nTidak ada pasien yang memenuhi kriteria filter.")

Memulai pemrosesan untuk file: patients.csv

--- Data Asli (DataFrame) ---
|   id | name     |   age | symptoms                        |
|-----:|:---------|------:|:--------------------------------|
|    1 | Andi     |    45 | demam, batuk, sesak napas       |
|    2 | Budi     |    29 | mual, sakit perut               |
|    3 | Citra    |    62 | pusing, kehilangan keseimbangan |
|    4 | Dita     |    30 | susah tidur                     |
|    5 | Eka      |    18 | gusi berdarah                   |
|    6 | Fitra    |    49 | pusing, sakit perut             |
|    7 | Gio      |    49 | menggigil, batuk, sakit kepala  |
|    8 | Harianto |    33 | memar di tangan                 |
|    9 | Idul     |    88 | susah tidur                     |
|   10 | Jaka     |    54 | sesak nafas                     |
----------------------------------------------------------------------
--- Data Setelah Tokenisasi Gejala ---
|   id | name     |   age | symptoms                        | symptom_l